In [1]:
%load_ext rqalpha

%rqalpha -s 20120101 -e 20171020 --account stock 100000 -fq 1m -p -bm 000001.XSHG

import head
import MeanVariance 
from datetime import timedelta
import talib
import pandas as pd
import numpy as np

def init(context):
    # 策略初始化运行
    context.high_market = 0
    context.opt_weights = []
    context.buylist = []
    context.selllist = []
    scheduler.run_monthly(rebalance,7)
    scheduler.run_daily(stop_loss)
    context.max_dropdown = 0.75 #最大回撤定为0.75
    context.prudent_model = 0 #慎重选股，突破回撤时启用
    #scheduler.run_daily(gold_in)
    #scheduler.run_daily(stop)

def stop_loss(context, bar_dict):
    
    
    # 市值低于建仓成本-5%的个股，卖出：
    for stock in context.portfolio.positions:
        #平均建仓成本
        average_price = context.portfolio.positions[stock].avg_price
        #市值
        market_value = context.portfolio.positions[stock].market_value
        
        #持股数
        quantity = context.portfolio.positions[stock].quantity
        if quantity != 0:
            stock_price = history_bars(stock,22,'1d',['close'],skip_suspended = False)
            stock_price = [x[0] for x in stock_price]
            #计算MA5,MA20
            average5 = talib.SMA(np.array(stock_price),5)
            average20 = talib.SMA(np.array(stock_price),20)
            average_rev = market_value/ quantity
            #止盈，收益大于80%
            if average_rev / average_price >= 1.8:
                print('止盈，卖出%s'%(stock))
                order_target_percent(stock, 0)
            #止损，市值低于建仓成本5%
            elif average_rev / average_price <= 0.95:
                print('止损，卖出%s'%(stock))
                order_target_percent(stock, 0)
    #如果空仓，选股
    #if len(context.portfolio.positions) == 0:
    #    reinvest(context)
    
#分钟数据，回撤处理
def drawdown_contral(context):
    total_market = context.portfolio.total_value
    if context.high_market < total_market:
        context.high_market = total_market
    #回撤25%，
    if total_market/context.high_market <= context.max_dropdown:
        #开启谨慎选股模式，双重选股，加入第二判断，净利率同比增长 >0 且不断增加
        context.prudent_model = 1
        
        #卖出pnl < 0的股票
        #累计盈亏
        for stock in context.portfolio.positions:
            if context.portfolio.positions[stock].pnl <0 and context.portfolio.positions[stock].quantity > 0:
                print('回撤控制，卖出：%s'%(stock))
                shares = context.portfolio.positions[stock].sellable
                if context.portfolio.positions[stock].quantity <= shares:
                    order_shares(stock, -context.portfolio.positions[stock].quantity)
                else:
                    order_shares(stock, -shares)
    else:
        context.prudent_model = 0
                
#选股票池,3个月选一次
def rebalance(context, bar_dict):
    
    #月初清仓
    if len(context.portfolio.positions)!= 0:
        for stock in context.portfolio.positions:
            order_target_percent(stock, 0)
            
    date = context.now + timedelta(-1)
    date = date.strftime('%Y%m%d')
    hy_and_stock, stock_list, peg_rs, peg_eps, list_buy = head.hy_run(date, fre = '2w', method= 'medium_15', inter_top= 5, hy_num= 5)
        #list_buy = head.bull_or_bear(date, flag = 'bull')
        #list_buy = head.cap_std(date)
        #dapan_flag = head.hushen_trend(date)
    
    context.buylist = [stock for stock in list_buy if not is_suspended(stock)]
    
    if len(context.buylist) > 1:
        data = get_price(context.buylist, start_date =head.get_pre_date(date, '1y') ,end_date = date, fields= ['close'])
        MV = MeanVariance.MeanVariance(data, context.buylist)
        context.buylist = MV.stock_list
        MV.opt()
        context.opt_weights = list(MV.opt_weights)
    else:
        context.opt_weights = [1.]
    #----------------加入谨慎选股判断------------------
    if context.prudent_model == 1:
        
        #为了防止谨慎选股重复开启，适当放宽条件，上调最大回撤，每次加0.05
        #context.max_dropdown -= 0.05
        print('开启谨慎选股模式')
        context.buylist = list(context.buylist)
        context.buylist = tuple(context.buylist)
        buy_list = list(context.buylist)
        
        for stock in context.buylist:
            if not head.prudent_choose(date, stock):
                buy_list.remove(stock)
                #context.buylist = buy_list
        
        context.buylist = buy_list
        
        if len(context.buylist) != 0:
            
            context.prudent_model = 0
            context.max_dropdown -= 0.05
            print('关闭谨慎选股模式,最大回撤上调为：', context.max_dropdown)
            
            if len(context.buylist) > 1:
                data = get_price(context.buylist, start_date =head.get_pre_date(date, '1y') ,end_date = date, fields= ['close'])
                MV = MeanVariance.MeanVariance(data, context.buylist)
                context.buylist = MV.stock_list
                MV.opt()
                context.opt_weights = list(MV.opt_weights)
            else:
                context.opt_weights = [1.]
                
            for i in range(len(context.buylist)):
                if context.opt_weights[i] != 0.:
                    if dead_not_in(context, context.buylist[i]) == 1:
                        order_target_percent(context.buylist[i], context.opt_weights[i])
                        logger.info('Buy %f %s' %(context.opt_weights[i], context.buylist[i]))
                    else:
                        print('死叉！！不操作%s'%(context.buylist[i]))
    #logger.info(list_buy, context.opt_weights)
    
    if len(context.buylist) != 0:
        for i in range(len(context.buylist)):
            if context.opt_weights[i] != 0.:
                if dead_not_in(context, context.buylist[i]) == 1:
                    order_target_percent(context.buylist[i], context.opt_weights[i])
                    logger.info('Buy %f %s' %(context.opt_weights[i], context.buylist[i]))
                else:
                    print('死叉！！不操作%s'%(context.buylist[i]))
    
    
#死叉判断进场，如果前3天有死叉，就不进场
def dead_not_in(context, stock):
    stock_price = history_bars(stock,20,'1d',['close'],skip_suspended = False)
            
    stock_price = [x[0] for x in stock_price]
    #print(stock_price)
            
    #计算MA5,MA10
    average5 = talib.SMA(np.array(stock_price[:]),5)
    average10 = talib.SMA(np.array(stock_price[:]),10)
            
    #10日双均线死叉不进场
    #if (average5[-1]<average10[-1]and average5[-2]>average10[-2]) or (average5[-2]<average10[-2]and average5[-3]>average10[-3]) or (average5[-3]<average10[-3]and average5[-4]>average10[-4]):
    #if (average5[-1]<average10[-1]and average5[-2]>average10[-2]) or (average5[-2]<average10[-2]and average5[-3]>average10[-3]):
    if (average5[-1]<average10[-1]and average5[-2]>average10[-2]):
        return 0
    else:
        return 1
    
def before_trading(context):
    pass
 
def handle_bar(context, bar_dict):
    # 每个 bar 数据运行
    drawdown_contral(context)
    
def after_trading(context):
    # 每日收盘后运行
    pass


ImportError: No module named head